# Advanced document indexing

# Splitting and ingesting HTML content

## Splitting and ingesting the content of a single URL (on Cornwall)

### Preparing the Chroma DB collections

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import getpass

OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

Enter your OPENAI_API_KEY ········


In [4]:
corwnall_granular_collection = Chroma( #A
    collection_name="cornwall_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_granular_collection.reset_collection() #B
# A Create a Chorma DB collection
# B Reset the collection in case it already exists 

In [5]:
corwnall_coarse_collection = Chroma( #A 
    collection_name="cornwall_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

corwnall_coarse_collection.reset_collection() #B
# A Create a Chorma DB collection
# B Reset the collection in case it already exists 

### Loading the HTML content with the AsyncHtmlLoader 

In [6]:
from langchain_community.document_loaders import AsyncHtmlLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
destination_url = "https://en.wikivoyage.org/wiki/Cornwall"

In [8]:
html_loader = AsyncHtmlLoader(destination_url)

In [9]:
docs = html_loader.load()

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  4.55it/s]


In [10]:
len(docs)

1

### Splitting into granular chunks with the HTMLSectionSplitter 

In [11]:
from langchain_text_splitters import HTMLSectionSplitter

In [12]:
headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]
html_section_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)

In [13]:
def split_docs_into_granular_chunks(docs):
    all_chunks = []
    for doc in docs:
        html_string = doc.page_content #A
        temp_chunks = html_section_splitter.split_text(html_string) #B
        all_chunks.extend(temp_chunks) 

    return all_chunks

#A Extract the HTML text from the document
#B Each chunk is a H1 or H2 HTML section

In [14]:
granular_chunks = split_docs_into_granular_chunks(docs)

#### Ingesting granular chunks

In [15]:
corwnall_granular_collection.add_documents(documents=granular_chunks)

['2feffdc6-aedf-4693-8c8e-7f8b18f520fb',
 '766f0004-adc5-4aab-a65e-96c7a7c03856',
 '201756b9-bb39-4d07-8926-93f546880e74',
 '9a12bc54-c1a6-42b4-b00e-2079708e8fee',
 'cc858ba3-2488-45a9-b47b-0514178bf3a0',
 '02d77c77-be27-4b3b-8eaf-119f48729ec1',
 '5a3ce6f2-2638-4983-ad03-9a1b7c9c45ad',
 '62425afa-d44c-4b23-bf4c-68276d9e1dab',
 '54f7f156-81d2-4151-a03d-906e8e38f315',
 '8ca62ce3-2ed7-468f-9810-abe2b3843473',
 '27f58e5c-047c-4cf3-9998-0ed82767618c',
 '8206881a-24fc-44f9-a841-38bfee4c16b1',
 '05bb64c9-f8f1-4e3d-a17b-ba775174181d',
 'cacad505-4713-4151-940a-05cbd82b4f95',
 'ec682f44-d3e9-408e-acb9-2463f6209d28',
 'c1363f6f-abdd-4ad4-9e17-c3802bed7807',
 'a9379d0b-bd11-4357-bc4e-4eca9e9e73ba',
 'c9e4fe8e-fc8e-4b3c-b9e0-93ee62266710',
 'a85b8924-8ccd-41d9-b1e1-c2f57cacd83c']

#### Searching granular chunks

In [16]:
results = corwnall_granular_collection.similarity_search(query="Events or festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='Do 
 [ edit ] 
 
 The  South West Coast Path  runs along the coastline of Britain’s south-west peninsula. The Cornish section is supposed to be the most scenic (unless you talk to someone in Devon, in which case the Devon part is most scenic). It is particularly scenic around Penwith and the Lizard. The trail takes walkers to busy towns, remote cliffs, beaches, heaths, farms and fishing villages. Walking along it is a great way to experience the region in all its variety. (Walking the entire path takes several weeks, walking on a choice part of it is easier.) 
 The  Camel Trail  is an  18-mile (29   km)  off-road cycle-track following the scenic estuary of the river Camel from  Padstow  to Wenford Bridge via  Wadebridge  and  Bodmin . 
 The  Cornish Film Festival  is held annually each November around  Newquay . 
 Cornwall, in particular Newquay, is the UK's  surfing  capital, with equipment hire and surf schools p

### Splitting into coarse chunks with the RecursiveCharacterTextSplitter 

In [17]:
from langchain_community.document_transformers import Html2TextTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [18]:
html2text_transformer = Html2TextTransformer()

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=300
)

In [20]:
def split_docs_into_coarse_chunks(docs):
    text_docs = html2text_transformer.transform_documents(docs) #A 
    coarse_chunks = text_splitter.split_documents(text_docs)

    return coarse_chunks
#A transform HTML docs into clean text docs

In [21]:
coarse_chunks = split_docs_into_coarse_chunks(docs)

#### Ingesting coarse chunks

In [22]:
corwnall_coarse_collection.add_documents(documents=coarse_chunks)

['1790ad15-f43b-49da-ada7-008f18649f7d',
 '2245d241-607b-4175-b6a9-6247f6c43f3d',
 'ab68f17e-d4ca-4562-ac3a-d3eb1b9ce802',
 'd833106e-0d7b-4231-9318-74a4181c3bec',
 'dfa4dc0b-0fc4-4fb5-bb34-43f3eb76c068',
 '5937741a-1611-418d-9fa0-c08da99cb4d3',
 '951c52f2-d2f6-46e7-b3bf-b6b6d0618c97',
 '45d254de-b72d-4898-aa6c-29d84b79ad94',
 '17023187-c33c-4c77-a21f-e4a90f08791b',
 '7c9c1bcd-d169-4020-bfc5-1a8bea0dc72c',
 'bb253ecf-530a-4ec3-b6b4-60a23f349530',
 '88c6ab07-a266-41fe-8261-7f8ed8421313',
 '0c0021b2-7538-452b-87f6-56be80749116',
 '52ed6dff-eb0e-4966-a04e-94d7b8003eb8']

#### Searching coarse chunks

In [23]:
results = corwnall_coarse_collection.similarity_search(query="Festivals in Cornwall",k=3)
for doc in results:
    print(doc)

page_content='### Spirits

[edit]

    _See also:Liquor_

Gin and rum are also produced in Cornwall. A popular brand of Cornish rum is
Dead Man's Fingers which has multiple flavours and is bottled in St. Ives.

## Festivals

[edit]

These festivals tend to not be public holidays and not all are celebrated
fully across the county.

AberFest. A Celtic cultural festival celebrating “All things” Cornish and
Breton that takes place biennially (every two years) in Cornwall at Easter.
The AberFest Festival alternates with the Breizh – Kernow Festival that is
held in Brandivy and Bignan (in Breizh/Bretagne – France) on the alternate
years. (updated Jun 2023)

**Golowan** , sometimes also _Goluan_ or _Gol-Jowan_ is the Cornish word for
the Midsummer celebrations, most popular in the Penwith area and in particular
Penzance and Newlyn. The celebrations are conducted from the 23rd of June (St
John's Eve) to the 28th of June (St Peter's Eve) each year, St Peter's Eve
being the more popular in Corni

## Splitting and ingesting the content of various URLs (across UK destinations)

### Preparing the Chroma DB collections

In [22]:
uk_granular_collection = Chroma( #A
    collection_name="uk_granular",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_granular_collection.reset_collection() #B

In [23]:
uk_coarse_collection = Chroma( #A
    collection_name="uk_coarse",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

uk_coarse_collection.reset_collection() #B

### Splitting and ingesting HTML content with the HTMLSectionSplitter 

In [24]:
uk_destinations = [
    "Cornwall", "North_Cornwall", "South_Cornwall", "West_Cornwall", 
    "Tintagel", "Bodmin", "Wadebridge", "Penzance", "Newquay",
    "St_Ives", "Port_Isaac", "Looe", "Polperro", "Porthleven"
    "East_Sussex", "Brighton", "Battle", "Hastings_(England)", 
    "Rye_(England)", "Seaford", "Ashdown_Forest"
]

wikivoyage_root_url = "https://en.wikivoyage.org/wiki"

In [25]:
uk_destination_urls = [f'{wikivoyage_root_url}/{d}' for d in uk_destinations]

In [26]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #C
    docs =  html_loader.load() #D
    
    for doc in docs:
        print(doc.metadata)
        granular_chunks = split_docs_into_granular_chunks(docs)
        uk_granular_collection.add_documents(documents=granular_chunks)

        coarse_chunks = split_docs_into_coarse_chunks(docs)
        uk_coarse_collection.add_documents(documents=coarse_chunks)
#A Create a Chorma DB collection
#B Reset the collection in case it already exists 
#C Loader for one destination
#D Documents of one destination 

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  9.17it/s]


{'source': 'https://en.wikivoyage.org/wiki/Cornwall', 'title': 'Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.60it/s]


{'source': 'https://en.wikivoyage.org/wiki/North_Cornwall', 'title': 'North Cornwall – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.05it/s]


...

{'source': 'https://en.wikivoyage.org/wiki/Rye_(England)', 'title': 'Rye (England) – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.74it/s]


{'source': 'https://en.wikivoyage.org/wiki/Seaford', 'title': 'Seaford – Travel guide at Wikivoyage', 'language': 'en'}


Fetching pages: 100%|####################################################################| 1/1 [00:01<00:00,  1.22s/it]


{'source': 'https://en.wikivoyage.org/wiki/Ashdown_Forest', 'title': 'Ashdown Forest – Travel guide at Wikivoyage', 'language': 'en'}


#### Searching 

In [27]:
granular_results = uk_granular_collection.similarity_search(query="Events or festivals in East Sussex",k=4)
for doc in granular_results:
    print(doc)

page_content='Brighton' metadata={'Header 1': 'Brighton'}
page_content='South Cornwall' metadata={'Header 1': 'South Cornwall'}
page_content='Seaford' metadata={'Header 1': 'Seaford'}
page_content='Penzance' metadata={'Header 1': 'Penzance'}


In [28]:
coarse_results = uk_coarse_collection.similarity_search(query="Events or festivals in East Sussex",k=4)
for doc in coarse_results:
    print(doc)

page_content='### Events

[edit]

A market during the Brighton Festival. The Theatre Royal is the red building.
A colourful parade down Queens Road during Pride in 2016.

  * **Brighton Racecourse** has flat-racing April-Oct. It's on Freshfield Rd a mile east of town centre.
  * **Plumpton Racecourse** is National Hunt (jumps races) Nov-March, but it's 10 mi (16 km) north in Lewes.
  * Brighton Festival Fringe: early May – early June, ☏ +44 1273 764900, info@brightonfringe.org. The Fringe runs at the same time as the main festival, and features over 600 events, including comedy, theatre, music, and "open houses" (local artists exhibiting in their own homes) and tours (haunted pubs, Regency Brighton, churches, cemeteries, sewers, etc.)_(date needs fixing)_
  * Brighton Festival: May, ☏ +44 1273 709709 (tickets), tickets@brightonfestival.org. The Brighton Festival, in May each year, is the second biggest arts festival in Great Britain (coming closely behind Edinburgh). Music of all sorts

In [29]:
granular_results = uk_granular_collection.similarity_search(query="Beaches in Conrwall",k=4)
for doc in granular_results:
    print(doc)

page_content='Cornwall' metadata={'Header 1': 'Cornwall'}
page_content='North Cornwall' metadata={'Header 1': 'North Cornwall'}
page_content='West Cornwall' metadata={'Header 1': 'West Cornwall'}
page_content='South Cornwall' metadata={'Header 1': 'South Cornwall'}


In [30]:
coarse_results = uk_coarse_collection.similarity_search(query="Beaches in Cornwall",k=4)
for doc in coarse_results:
    print(doc)

page_content='## Understand

[edit]

The north coast on the Celtic Sea, part of the Atlantic Ocean, is more exposed
than the south coast, and therefore has a wilder nature. The High Cliff,
between Boscastle and St Gennys, is the highest sheer-drop cliff in Cornwall
at 223 metres (732 ft).

Beaches, which form an important part of the tourist industry, include Bude,
Polzeath, Watergate Bay, Perranporth, Porthtowan, Fistral Beach, Newquay, St
Agnes, St Ives, and on the south coast Gyllyngvase beach in Falmouth and the
large beach at Praa Sands further to the south-west.

## Get in

[edit]

### By train

[edit]

Regular trains run on the main line from London Paddington (8 daily all the
way through Cornwall to Penzance, 5 hr 30 min), Bristol, Birmingham etc. to
Penzance. There is also an overnight sleeper train which runs Su-F nights from
London Paddington and Penzance.

### By car

[edit]

Cornwall can be accessed by road via the A30 which runs from the end of the M5
at Exeter, all the w

# Embedding strategy

## Embedding child chunks with ParentDocumentRetriever

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Setting up the Parent Document retriever

In [42]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_store = InMemoryStore() #E

parent_doc_retriever = ParentDocumentRetriever( #F
    vectorstore=child_chunks_collection,
    docstore=doc_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [43]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    print(f'Ingesting {destination_url}')
    parent_doc_retriever.add_documents(text_docs, ids=None) #D

#A Loader for destination web page
#B HTML documents of one destination 
#C Transform HTML docs into clean text deocs
#D Ingest coarse chunks into document store and granular chunks into vector store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.85it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.51it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  1.81it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  5.26it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  3.83it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [44]:
# list(doc_store.yield_keys()) #A <===IMPORTANT: reenable before pushing code in
#A Show the keys of the added coarse chunks

### Performing a search on granular information 

In [45]:
retrieved_docs = parent_doc_retriever.invoke("Cornwall Ranger")

In [46]:
len(retrieved_docs)

4

In [47]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All D

### Comparing with direct semantic search on child chunks

In [48]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [49]:
len(child_docs_only)

4

In [50]:
child_docs_only[0]

Document(metadata={'doc_id': '03d8e7e2-27d4-4921-8457-2487408a4c72', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [51]:
# IMPORTANT: as you can see a granular search would have identified the chunk, but it would have lost the usefulcontext about travelling in Cornwall

## Embedding child chunks with MultiVectorRetriever

In [52]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [53]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #B

child_chunks_collection = Chroma( #C
    collection_name="uk_child_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

child_chunks_collection.reset_collection() #D

doc_byte_store = InMemoryByteStore() #E
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #F
    vectorstore=child_chunks_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Splitter to generate child granular chunks from parent coarse chunks
#C Vector store collection to host child granular chunks
#D Make sure the collection is empty
#E Document store to host parent coarse chunks
#F Retriever to link parent coarse chunks to child granular chunks

### Ingesting the content into doc and vector store

In [54]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_granular_chunks = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        granular_chunks = child_splitter.split_documents([coarse_chunk]) #F

        for granular_chunk in granular_chunks:
            granular_chunk.metadata[doc_key] = coarse_chunk_id #G

        all_granular_chunks.extend(granular_chunks)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_granular_chunks) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into parent coarse chunks
#E Iterate over the parent coarse chunks
#F Create child granular chunks form each parent coarse chunk
#G Link each child granular chunk to its parent coarse chunk
#H Ingest the child granular chunks into the vector store
#I Ingest the parent coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.63it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  5.44it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.44it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  7.35it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.01it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [55]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [56]:
len(retrieved_docs)

4

In [57]:
retrieved_docs[0]

Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All D

In [58]:
##IMPORTANT: same as PArent Document retriever, but more control and flexibility on how to link child to parent chunks

### Comparing with direct semantic search on child chunks

In [59]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [60]:
len(child_docs_only)

4

In [61]:
child_docs_only[0]

Document(metadata={'doc_id': '631e7780-e23d-4ca9-83fb-b0be24091611', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [62]:
## IMPORTANT: Same as before

## Embedding summaries with MultiVectorRetriever

In [63]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid

### Setting up the Multi vector retriever (similar to when embedding child chunks)

In [64]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

summaries_collection = Chroma( #B
    collection_name="uk_summaries",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

summaries_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=summaries_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the summarization chain

In [65]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

In [66]:
summarization_chain = (
    {"document": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{document}") #B
    | llm
    | StrOutputParser())

#A Grab the text content from the document
#B Instantiate a prompt asking to generate summary of the provided text
#C Send the LLM the instantiated prompt 
#D Extract the summary text from the response

### Ingesting the coarse chunks and related summaries into doc and vector store

In [67]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_summaries = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        summary_text =  summarization_chain.invoke(coarse_chunk) #F
        summary_doc = Document(page_content=summary_text, metadata={doc_key: coarse_chunk_id})

        all_summaries.append(summary_doc) #G

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_summaries) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a summary for the coarse chunk thorugh the summarization chain
#G Link each summary to its related coarse chunk
#H Ingest the summaries into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.83it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 12.04it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.53it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  2.94it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.85it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


In [70]:
# COMMENT: the code above is similar to when ingesting child chunks, but it is slower because of the summarization step
# which invokes the LLM.
# The processing can be speeded up by parallelizing the outer for loop on the destination urls.

### Performing a search on granular information

In [71]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall travel")

In [72]:
len(retrieved_docs)

4

In [73]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage', 'language': 'en'}, page_content="Trains from London take about 3 hr 20 min to Plymouth.\n\n### By car\n\n[edit]\n\nCornwall can be accessed by road via the A30 which runs from the end of the M5\nat Exeter, all the way through the heart of Devon and Cornwall down to Land's\nEnd. It is a grade-separated expressway as far as Carland Cross near Truro\n(the expressway is expected to be open as far as Camborne (between Redruth and\nHayle) by March 2024). You can also get to Cornwall via the A38, crossing the\nRiver Tamar at Plymouth via the Tamar Bridge, which levies a toll on eastbound\nvehicles. On summer Saturdays and during bank holiday weekends roads to\nCornwall are usually busy.\n\n## Get around\n\n[edit]\n\n### By bus\n\n[edit]\n\nThanks to Transport for Cornwall, all bus tickets are interchangeable across\nthe different companies. The **Cornwall All 

### Comparing with direct semantic search on summaries

In [74]:
summary_docs_only =  summaries_collection.similarity_search("Cornwall Travel")

In [75]:
len(summary_docs_only)

4

In [76]:
summary_docs_only

[Document(metadata={'doc_id': '68673c9f-4ae9-4026-8f73-abd7218557f6'}, page_content="The document is a travel guide excerpt from Wikivoyage about Cornwall, a county in the southwest of the United Kingdom. It describes Cornwall as a distinct and popular tourist destination known for its warm climate, long coastline, stunning scenery, and rich Celtic heritage, including legends of pirates and King Arthur. The region is recognized for its cultural tourism and archaeological significance, with over 30% of the area designated as an Area of Outstanding Natural Beauty (AONB).\n\nThe guide outlines various sections, including regions, towns and cities, visitor information, transportation options, attractions like National Trust properties, local cuisine, beverages, festivals, accommodation, and safety tips. Cornwall is divided into three main regions: North Cornwall, South Cornwall, and West Cornwall, each offering unique landscapes and experiences. The document emphasizes the area's pride in 

In [77]:
# COMMENT: a direct search on summaries retrieves denser information, but it is missing out on useful details. 
# However, you might consider using the summaries directly if after testing they prove adequate.

## Embedding hypotetical questions with MultiVectorRetriever

In [78]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import uuid
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

### Setting up the Multi vector retriever (same as when embedding summaries)

In [79]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000) #A

hypotetical_questions_collection = Chroma( #B
    collection_name="uk_hypotetical_questions",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

hypotetical_questions_collection.reset_collection() #C

doc_byte_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=hypotetical_questions_collection,
    byte_store=doc_byte_store
)
#A Splitter to generate parent coarse chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host parent coarse chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Setting up the chain to generate hypotetical questions

In [80]:
class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions for given text."""

    questions: List[str] = Field(..., description="List of hypotetical questions for given text")

In [81]:
llm_with_structured_output = ChatOpenAI(model="gpt-4o-mini", 
        openai_api_key=OPENAI_API_KEY).with_structured_output(
        HypotheticalQuestions
)

In [82]:
hypotetical_questions_chain = (
    {"document_text": lambda x: x.page_content} #A
    | ChatPromptTemplate.from_template( #B
        "Generate a list of exactly 4 hypothetical questions that the below text could be used to answer:\n\n{document_text}"
    )
    | llm_with_structured_output #C
    | (lambda x: x.questions) #D
)

#A Grab the text content from the document
#B Instantiate a prompt asking to generate 4 hypotetical questions on the provided text
#C Invoke the LLM configured to return an object containing the questions as a typed list of strings
#D Grab the list of questions from the response

### Ingesting the coarse chunks and related hypotetical questions into doc and vector store

In [83]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    coarse_chunks = parent_splitter.split_documents(text_docs) #D

    coarse_chunks_ids = [str(uuid.uuid4()) for _ in coarse_chunks]
    all_hypotetical_questions = []
    for i, coarse_chunk in enumerate(coarse_chunks): #E
        
        coarse_chunk_id = coarse_chunks_ids[i]
            
        hypotetical_questions = hypotetical_questions_chain.invoke(coarse_chunk) #F
        hypotetical_questions_docs = [Document(page_content=question, metadata={doc_key: coarse_chunk_id})
                                              for question in hypotetical_questions] #G

        all_hypotetical_questions.extend(hypotetical_questions_docs)

    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(all_hypotetical_questions) #H
    multi_vector_retriever.docstore.mset(list(zip(coarse_chunks_ids, coarse_chunks))) #I

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into coarse chunks
#E Iterate over the coarse chunks
#F Generate a list of hypotetical questions for the coarse chunk thorugh the question generation chain
#G Link each hypotetical question to its related coarse chunk
#H Ingest the hypotetical questions into the vector store
#I Ingest the coarse chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.62it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.77it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  9.71it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 11.38it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.48it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [84]:
retrieved_docs = multi_vector_retriever.invoke("How can you go to Brighton from London?")

In [85]:
len(retrieved_docs)

4

In [86]:
retrieved_docs

[Document(metadata={'source': 'https://en.wikivoyage.org/wiki/Brighton', 'title': 'Brighton – Travel guide at Wikivoyage', 'language': 'en'}, page_content='There are particular days of the year when it is **very much** inadvisable to\ndrive into Brighton:\n\n  * The children\'s parade day at the start of Brighton Festival. Usually the first Saturday in May. Many roads in the centre of Brighton are closed.\n  * The day of the annual London to Brighton Bike Ride. This is on a Sunday in June - tens of thousands of cyclists plus their support vehicles are in the city, so many roads will be blocked or difficult to get across.\n  * The parade day of the Brighton and Hove Pride week. Around the first Saturday of August. Many roads in the centre of Brighton and around the pier area are closed to all traffic, and diversionary routes are long and/or not built for heavy traffic. Gridlock often ensues on Pride Saturday.\n  * The Brighton Marathon in early April, where many roads in the city are cl

### Inspecting possible questions matching our question through semantic search

In [87]:
hypotetical_question_docs_only =  hypotetical_questions_collection.similarity_search("How can you go to Brighton from London?")

In [88]:
len(hypotetical_question_docs_only)

4

In [89]:
hypotetical_question_docs_only

[Document(metadata={'doc_id': 'e5ce4138-5d20-40dd-8410-42eab6aa221c'}, page_content='If a tourist planned to visit Brighton on a sunny summer day, what alternative transportation options could they consider to avoid traffic?'),
 Document(metadata={'doc_id': '363732e5-c50b-4206-b218-40283dd42a7f'}, page_content='What if someone wanted to visit multiple attractions in Brighton?'),
 Document(metadata={'doc_id': '0d82094a-f5bb-4bfa-b982-799c43f7d008'}, page_content='How might transportation options evolve in Brighton if more people begin commuting from London daily?'),
 Document(metadata={'doc_id': 'a83b93ea-7ff3-4b20-8918-d4113febc10e'}, page_content='How would travel times differ for someone choosing to go to London by train versus by coach from Brighton?')]

# Granular chunk expansion with MultiVectorRetriever

In [90]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import uuid

### Setting up the Multi vector retriever

In [91]:
granular_chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=500) #A

granular_chunks_collection = Chroma( #B
    collection_name="uk_granular_chunks",
    embedding_function=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

granular_chunks_collection.reset_collection() #C

expanded_chunk_store = InMemoryByteStore() #D
doc_key = "doc_id"

multi_vector_retriever = MultiVectorRetriever( #E
    vectorstore=granular_chunks_collection,
    byte_store=expanded_chunk_store
)
#A Splitter to generate granular chunks from original documents (parsed from web pages)
#B Vector store collection to host child granular chunks
#C Make sure the collection is empty
#D Document store to host expanded chunks
#E Retriever to link parent coarse chunks to child granular chunks

### Ingesting granular and expanded chunks into doc and vector store

In [92]:
for destination_url in uk_destination_urls:
    html_loader = AsyncHtmlLoader(destination_url) #A
    html_docs =  html_loader.load() #B
    text_docs = html2text_transformer.transform_documents(html_docs) #C

    granular_chunks = granular_chunk_splitter.split_documents(text_docs) #D

    expanded_chunk_store_items = []
    for i, granular_chunk in enumerate(granular_chunks): #E

        this_chunk_num = i #F
        previous_chunk_num = i-1 #F
        next_chunk_num = i+1 #F
        
        if i==0: #F
            previous_chunk_num = None
        elif i==(len(granular_chunks)-1): #F
            next_chunk_num = None

        expanded_chunk_text = "" #G
        if previous_chunk_num: #G
            expanded_chunk_text += granular_chunks[previous_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_text += granular_chunks[this_chunk_num].page_content #G
        expanded_chunk_text += "\n"

        if next_chunk_num: #G
            expanded_chunk_text += granular_chunks[next_chunk_num].page_content
            expanded_chunk_text += "\n"

        expanded_chunk_id = str(uuid.uuid4()) #H
        expanded_chunk_doc = Document(page_content=expanded_chunk_text) #I

        expanded_chunk_store_item = (expanded_chunk_id, expanded_chunk_doc)
        expanded_chunk_store_items.append(expanded_chunk_store_item)

        granular_chunk.metadata[doc_key] = expanded_chunk_id #J
            
    print(f'Ingesting {destination_url}')
    multi_vector_retriever.vectorstore.add_documents(granular_chunks) #K
    multi_vector_retriever.docstore.mset(expanded_chunk_store_items) #L

#A Loader for one destination
#B Documents of one destination 
#C transform HTML docs into clean text docs
#D Split the destination content into granular chunks
#E Iterate over the granular chunks
#F determine the index of the current chunk and its previous and next chunks
#G Assemble the text of the expanded chunk by including the previous and next chunk
#H Generate the ID of the expanded chunk
#I Create the expanded chunk document
#J Link each granular chunk to its related expanded chunk
#K Ingest the granular chunks into the vector store
#L Ingest the expanded chunks into the document store

Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.26it/s]


Ingesting https://en.wikivoyage.org/wiki/Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  6.04it/s]


Ingesting https://en.wikivoyage.org/wiki/North_Cornwall


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.91it/s]


...

Ingesting https://en.wikivoyage.org/wiki/Rye_(England)


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00,  8.13it/s]


Ingesting https://en.wikivoyage.org/wiki/Seaford


Fetching pages: 100%|####################################################################| 1/1 [00:00<00:00, 10.64it/s]


Ingesting https://en.wikivoyage.org/wiki/Ashdown_Forest


### Performing a search on granular information

In [193]:
retrieved_docs = multi_vector_retriever.invoke("Cornwall Ranger")

In [194]:
len(retrieved_docs)

4

In [195]:
retrieved_docs[0]

Document(page_content="Buses only serve designated stops when in towns; otherwise, you can flag them\ndown anywhere that's safe for them to stop.\n\n### By train\n\n[edit]\n\n**CrossCountry Trains** and **Great Western Railway** operate regular train\nservices between the main centres of population, the latter company also\nserving a number of other towns on branch lines. For train times and fares\nvisit National Rail Enquiries.\nThe **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.\n\nThe **Lost Gardens of

### Comparing with direct semantic search on granular chunks

In [196]:
child_docs_only =  child_chunks_collection.similarity_search("Cornwall Ranger")

In [197]:
len(child_docs_only)

4

In [198]:
child_docs_only[0]

Document(metadata={'doc_id': '04c7f88e-e090-4057-af5b-ea584e777b3f', 'language': 'en', 'source': 'https://en.wikivoyage.org/wiki/South_Cornwall', 'title': 'South Cornwall – Travel guide at Wikivoyage'}, page_content='The **Cornwall Ranger** ticket allows unlimited train travel in Cornwall and\nPlymouth for a calendar day. As of 2023, this costs £14 for adults and £7 for\nunder-16s.\n\n## See\n\n[edit]\n\nThe **Eden Project** , near St Austell, a fabulous collection of flora from\nall over the planet housed in two space age transparent domes, and a massive\nzip line.')

In [ ]:
# COMMENT: the expanded chunk has more useful context